# 预测药物MoA
一个样本可以有为一个或多个MoA标记，属于多标签分类任务(MultiLabel Classification)

kaggle: https://www.kaggle.com/competitions/lish-moa/overview

## Import

In [ ]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## Data

EDA refences: https://www.kaggle.com/code/amiiiney/drugs-moa-classification-eda/notebook#1-Overview:-Features

In train_features: 
- Features **g-** signify gene expression data.

- Features **c-** signify cell viability data.
 
- **cp_type** indicates samples treated with a compound, **trt_cp** samples treated with the compounds.**ctl_vehicle**; control perturbations have no MoAs.

- **cp_time** and **cp_dose** indicate treatment duration (24, 48, 72 hours) and dose (high or low).

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_noscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

test_features = pd.read_csv('../input/lish-moa/test_features.csv')

train_features.head()

In [ ]:
train_features.shape, train_targets_scored.shape, train_targets_noscored.shape, test_features.shape

In [ ]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

len(GENES), len(CELLS)

In [ ]:
train_features['cp_type'].unique()

In [ ]:
target_count = train_targets_scored.sum()[1:].sort_values()
sns.set_style('whitegrid')
sns.histplot(target_count)

## Preprocess

In [ ]:
def map(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 1, 'ctl_vehicle': 0})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train_features = map(train_features)
test_features = map(test_features)

### PCA features + Existing features

In [ ]:
# GENE
n_comp = 50

data = pd.concat([train_features[GENES], test_features[GENES]])
data2 = PCA(n_components=n_comp, random_state=42).fit_transform(data)

train2 = data2[:len(train_features)]
test2 = data2[len(train_features):]

train2 = pd.DataFrame(train2, columns=[f'g-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'g-{i}' for i in range(n_comp)])

train_features = pd.concat([train_features, train2], axis=1)
test_features = pd.concat([test_features, test2], axis=1)

train_features.shape, test_features.shape

In [ ]:
# CELL
n_comp = 15

data = pd.concat([train_features[CELLS], test_features[CELLS]])
# PCA 返回的是numpy array
data2 = PCA(n_components=n_comp, random_state=42).fit_transform(data)

train2 = data2[:len(train_features)]
test2 = data2[len(train_features):]

train2 = pd.DataFrame(train2, columns=[f'c-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'c-{i}' for i in range(n_comp)])

train_features = pd.concat([train_features, train2], axis=1)
test_features = pd.concat([test_features, test2], axis=1)

train_features.shape, test_features.shape

### feature selection by VarianceThreshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(threshold=0.5)
data = pd.concat([train_features, test_features])
# transform 返回的是numpy array
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_transformed = data_transformed[:len(train_features)]
test_transformed = data_transformed[len(train_features):]

train_features = pd.concat([train_features.iloc[:, :4], pd.DataFrame(train_transformed)], axis=1)
test_features = pd.concat([test_features.iloc[:, :4], pd.DataFrame(test_transformed)], axis=1)

train_features.shape, test_features.shape

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')

train = train[train_features['cp_type'] == 1].reset_index(drop=True)
test = test_features[test_features['cp_type'] == 1].reset_index(drop=True)
target = train[train_targets_scored.columns]

train.drop('cp_type', axis=1, inplace=True)
test.drop('cp_type', axis=1, inplace=True)

train.shape, test.shape, target.shape

### Distribution plots

In [ ]:
# plt.figure(figsize=(16, 16))
# gene_choice = np.random.choice(len(GENES), 16)

# for i, col in enumerate(gene_choice):
#     plt.subplot(4, 4, i + 1)
#     plt.hist(train_features.loc[:, col], bins=100, color='orange')
#     plt.title(GENES[col])

### Cross Validation

由于样本的类别数存在不平衡，普通`Kfold`随机分类可能会导致训练集标签全为某个单一类别，而`StratifiedKFold`会按比例进行划分。
但是scikit-learn的StratifiedKFold不能用于多标签分类

In [ ]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=5)
    
for f, (train_idx, valid_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[valid_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)
folds

In [ ]:
target_col = target.drop('sig_id', axis=1).columns.values.tolist()
feature_col = [col for col in train.columns if col not in target_col]
feature_col = [col for col in feature_col if col not in ['sig_id']]

print(train.shape, test.shape, target.shape, folds.shape)
print(len(feature_col), len(target_col))

## Dataset

In [ ]:
class MoADataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    
    def __getitem__(self, idx):
        return {
            'x': torch.tensor(self.features[idx, :], dtype=torch.float32),
            'y': torch.tensor(self.targets[idx, :], dtype=torch.float32)
        }

    def __len__(self):
        return len(self.features)

class TestDataset(Dataset):
    def __init__(self, features):
        self.features = features
    
    def __getitem__(self, idx):
        return {
            'x': torch.tensor(self.features[idx, :], dtype=torch.float32)
        }

    def __len__(self):
        return len(self.features)

## Training functions

In [ ]:
def train_fn(model, optimizer, scheduler, criterion, dataloader, device):
    model.train()
    final_loss = 0

    for data in dataloader:
        optimizer.zero_grad()
        input, target = data['x'].to(device), data['y'].to(device)
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)

        final_loss += loss.item()

    return final_loss / len(dataloader)

def valid_fn(model, dataloader, criterion, device):
    model.eval()
    final_loss = 0
    valid_preds = []

    for data in dataloader:
        input, target = data['x'].to(device), data['y'].to(device)
        output = model(input)
        loss = criterion(output, target)

        final_loss += loss.item()
        valid_preds.append(output.sigmoid().detach().cpu().numpy())

    final_loss = final_loss / len(dataloader)
    valid_preds = np.concatenate(valid_preds)   # list -> numpy array(matrix)
    return final_loss , valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    final_preds = []

    with torch.no_grad():
        for data in dataloader:
            input = data['x'].to(device)
            output = model(input)
            final_preds.append(output.sigmoid().detach().cpu().numpy())

    final_preds = np.concatenate(final_preds)
    return final_preds

## Model

In [ ]:
class Model(nn.Module):
    def __init__(self, num_features, hidden_size, num_targets):
        super().__init__()
        self.model = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(0.2),
            nn.Linear(num_features, hidden_size),
            nn.LeakyReLU(),

            # nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            
            # nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),

            # nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),

            # nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),

            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, num_targets),
        )

    def forward(self, x):
        return self.model(x)

## Single Fold Training

In [ ]:
# Hyperparameters
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
epochs = 30
batch_size = 1024
learning_rate = 1e-3
weight_decay = 1e-5
nfolds = 5
early_stop_step = 10
early_stop = False

num_features = len(feature_col)
hidden_size = 2048
num_targets = len(target_col)

In [ ]:
def run_training(fold):
    train_idx = train[folds['kfold'] != fold].index
    valid_idx = train[folds['kfold'] == fold].index


    train_df = train[folds['kfold'] != fold].reset_index(drop=True)
    valid_df = train[folds['kfold'] == fold].reset_index(drop=True)

    x_train, y_train = train_df[feature_col].values, train_df[target_col].values
    x_valid, y_valid = valid_df[feature_col].values, valid_df[target_col].values

    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    model = Model(num_features, hidden_size, num_targets).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
    criterion = nn.BCEWithLogitsLoss()

    best_loss = np.inf
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))

    for epoch in range(epochs):
        train_loss = train_fn(model, optimizer, scheduler, criterion, train_dataloader, device)
        valid_loss, valid_preds = valid_fn(model, valid_dataloader, criterion, device)

        if valid_loss < best_loss:
            best_loss = valid_loss
            early_stop = 0
            torch.save(model.state_dict(), f'FOLD{fold}_.pth')
        else:
            early_stop += 1

        if early_stop >= early_stop_step:
            print(f'Early stop at epoch {epoch + 1}')
            break

        print(f'Fold {fold} | Epoch {epoch + 1} | train loss: {train_loss:.4f} | valid loss: {valid_loss:.4f}')

    # <-------------- inference ---------------->/
    x_test = test[feature_col].values
    test_dataset = TestDataset(x_test)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model = Model(num_features, hidden_size, num_targets)
    model.load_state_dict(torch.load(f'FOLD{fold}_.pth'))
    model.to(device)

    predictions = inference_fn(model, test_dataloader, device)

    return oof, predictions

In [ ]:
def run_k_fold(nfolds):
    oof = np.zeros((len(train), len(target_col)))
    predictions = np.zeros((len(test), len(target_col)))

    for fold in range(nfolds):
        oof_, pred_ = run_training(fold)
        oof += oof_
        predictions += pred_ / nfolds

    return oof, predictions

In [ ]:
oof, predictions = run_k_fold(nfolds)
# 将输出存入原dataframe的target列
train[target_col] = oof
test[target_col] = predictions

## Submit

In [ ]:
train_targets_scored.shape, len(target_col)

In [ ]:
valid_results = train_targets_scored.drop(columns=target_col).merge(train[['sig_id']+target_col], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_col].values
y_pred = valid_results[target_col].values

score = 0
for i in range(len(target_col)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

In [ ]:
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
sub = sample_submission.drop(columns=target_col).merge(test[['sig_id']+target_col], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)